In [ ]:
import tensorflow as tf
import sounddevice as sd
import numpy as np
import IPython.display as ipd

In [ ]:
tf.config.set_visible_devices(tf.config.list_physical_devices()[0])
tf.config.list_logical_devices()

In [ ]:
sd.default.samplerate = 44100
sd.default.channels = 1

In [ ]:
path_to_model = 'models/conv/MiraConv-v1.3.h5'
model = tf.keras.models.load_model(path_to_model)
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.BinaryCrossentropy(),
    metrics=[
        tf.keras.metrics.Recall(name='recall'),
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.AUC(name='auc'),
        tf.keras.metrics.BinaryAccuracy(),
    ]
)
model.summary()

In [ ]:
data = sd.rec(int(1.0 * sd.default.samplerate), blocking=True)

In [ ]:
def callback(indata, outdata, frames, time, status):
    # outdata[:] = indata
    global data, model
    data = data[frames:]
    data = np.append(data, indata, axis=0)
    prediction = model(np.reshape(data, (1, *data.shape)),training=False)[0][0].numpy()
    ipd.clear_output(wait=True)
    if prediction > 0.99:
        sd.play(data)
        print('Mira {:.3f}'.format(prediction))
    else:
        ipd.clear_output()
    
block = int(sd.default.samplerate * 0.1)
stream = sd.Stream(blocksize=block, callback=callback)

In [ ]:
stream.start()

In [ ]:
stream.stop()

In [ ]:
data.shape